In [1]:
from modularity_aware_gae.evaluation import community_detection, link_prediction
from modularity_aware_gae.input_data import load_data, load_labels
from modularity_aware_gae.louvain import louvain_clustering
from modularity_aware_gae.model import  GCNModelAE, GCNModelVAE, LinearModelAE, LinearModelVAE
from modularity_aware_gae.optimizer import OptimizerAE, OptimizerVAE
from modularity_aware_gae.preprocessing import *
from modularity_aware_gae.sampling import get_distribution, node_sampling
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Parameters related to the input data
flags.DEFINE_string('dataset', 'wiki', 'Graph dataset, among: cora, citeseer, wiki, celegans, email, polbooks, texas, wisconsin')

flags.DEFINE_boolean('features', False, 'Whether to include node features')


# Parameters related to the Modularity-Aware GAE/VGAE model to train

# 1/3 - General parameters associated with GAE/VGAE
flags.DEFINE_string('model', 'linear_vae', 'Model to train, among: gcn_ae, gcn_vae, \
                                            linear_ae, linear_vae')
flags.DEFINE_float('dropout', 0., 'Dropout rate')
flags.DEFINE_integer('iterations', 200, 'Number of iterations in training')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (Adam)')
flags.DEFINE_integer('hidden', 32, 'Dimension of the GCN hidden layer')
flags.DEFINE_integer('dimension', 16, 'Dimension of the output layer, i.e., \
                                       dimension of the embedding space')

# 2/3 - Additional parameters, specific to Modularity-Aware models
flags.DEFINE_float('beta', 0.0, 'Beta hyperparameter of Mod.-Aware models')
flags.DEFINE_float('lamb', 0.0, 'Lambda hyperparameter of Mod.-Aware models')
flags.DEFINE_float('gamma', 1.0, 'Gamma hyperparameter of Mod.-Aware models')
flags.DEFINE_integer('s_reg', 2, 's hyperparameter of Mod.-Aware models')

# 3/3 - Additional parameters, aiming to improve scalability
flags.DEFINE_boolean('fastgae', False, 'Whether to use the FastGAE framework')
flags.DEFINE_integer('nb_node_samples', 1000, 'In FastGAE, number of nodes to \
                                               sample at each iteration, i.e., \
                                               size of decoded subgraph')
flags.DEFINE_string('measure', 'degree', 'In FastGAE, node importance measure used \
                                          for sampling: degree, core, or uniform')
flags.DEFINE_float('alpha', 1.0, 'alpha hyperparameter associated with the FastGAE sampling')
flags.DEFINE_boolean('replace', False, 'Sample nodes with or without replacement')
flags.DEFINE_boolean('simple', True, 'Use simpler (and faster) modularity in optimizers')


# Parameters related to the experimental setup
flags.DEFINE_string('task', 'task_2', 'task_1: pure community detection \
                                       task_2: joint link prediction and \
                                               community detection')
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 1., 'Proportion of edges in validation set \
                                    for the link prediction task')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      for the link prediction task')
flags.DEFINE_boolean('validation', False, 'Whether to compute validation \
                                           results at each iteration, for \
                                           the link prediction task')
flags.DEFINE_boolean('verbose', True, 'Whether to print all comments details')


# Introductory message
if FLAGS.verbose:
    introductory_message()


 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: wiki
- Mode name: linear_vae
- Number of models to train: 10
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the output layer: 16
- lambda: 0.0
- beta: 0.0
- gamma: 1.0
- s: 2
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 



In [3]:
# Initialize lists to collect final results
mean_ami = []
mean_ari = []

mean_roc = []
mean_ap = []

# Load data
if FLAGS.verbose:
    print("LOADING DATA\n")
    print("Loading the", FLAGS.dataset, "graph")
adj_init, features_init = load_data(FLAGS.dataset)
#labels = load_labels(adj_init.shape[0])
if FLAGS.verbose:
    print("- Number of nodes:", adj_init.shape[0])
    #print("- Number of communities:", len(np.unique(labels)))
    print("- Use of node features:", FLAGS.features)
    print("Done! \n \n \n \n")

LOADING DATA

Loading the wiki graph
- Number of nodes: 2405
- Use of node features: False
Done! 
 
 
 



In [4]:
# We repeat the entire training+test process FLAGS.nb_run times
for i in range(FLAGS.nb_run):
    if FLAGS.verbose:
        print("EXPERIMENTS ON MODEL", i + 1, "/", FLAGS.nb_run, "\n")
        print("STEP 1/3 - PREPROCESSING STEPS \n")


    # Edge masking for Link Prediction:
    if FLAGS.task == 'task_2':
        # Compute Train/Validation/Test sets
        if FLAGS.verbose:
            print("Masking some edges from the training graph, for link prediction")
            print("(validation set:", FLAGS.prop_val, "% of edges - test set:",
                  FLAGS.prop_test, "% of edges)")
        #adj, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)
        adj, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        adj = adj_init

    # Compute the number of nodes
    num_nodes = adj.shape[0]

    # Preprocessing on node features
    if FLAGS.verbose:
        print("Preprocessing node features")
    if FLAGS.features:
        features = features_init
    else:
        # If features are not used, replace feature matrix by identity matrix
        features = sp.identity(num_nodes)
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
    if FLAGS.verbose:
        print("Done! \n")


    # Community detection using Louvain, as a preprocessing step
    if FLAGS.verbose:
        print("Running the Louvain algorithm for community detection")
        print("as a preprocessing step for the encoder")
    # Get binary community matrix (adj_louvain_init[i,j] = 1 if nodes i and j are
    # in the same community) as well as number of communities found by Louvain
    adj_louvain_init, nb_communities_louvain, partition = louvain_clustering(adj, FLAGS.s_reg)
    if FLAGS.verbose:
        print("Done! Louvain has found", nb_communities_louvain, "communities \n")

    # FastGAE: node sampling for stochastic subgraph decoding - used when facing scalability issues
    if FLAGS.fastgae:
        if FLAGS.verbose:
            print("Preprocessing operations related to the FastGAE method")
        # Node-level p_i degree-based, core-based or uniform distribution
        node_distribution = get_distribution(FLAGS.measure, FLAGS.alpha, adj)
        # Node sampling for initializations
        sampled_nodes, adj_label, adj_sampled_sparse = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        sampled_nodes = np.array(range(FLAGS.nb_node_samples))


    # Placeholders
    if FLAGS.verbose:
        print('Setting up the model and the optimizer')
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_layer2': tf.sparse_placeholder(tf.float32), # Only used for 2-layer GCN encoders
        'degree_matrix': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ()),
        'sampled_nodes': tf.placeholder_with_default(sampled_nodes, shape = [FLAGS.nb_node_samples])
    }


    # Create model
    if FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    elif FLAGS.model == 'gcn_ae':
        # 2-layer GCN Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # 2-layer GCN Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    else:
        raise ValueError('Undefined model!')


    # Optimizer
    if FLAGS.fastgae:
        num_sampled = adj_sampled_sparse.shape[0]
        sum_sampled = adj_sampled_sparse.sum()
        pos_weight = float(num_sampled * num_sampled - sum_sampled) / sum_sampled
        norm = num_sampled * num_sampled / float((num_sampled * num_sampled - sum_sampled) * 2)
    else:
        pos_weight = float(num_nodes * num_nodes - adj.sum()) / adj.sum()
        norm = num_nodes * num_nodes / float((num_nodes * num_nodes - adj.sum()) * 2)

    if FLAGS.model in ('gcn_ae', 'linear_ae'):
        opt = OptimizerAE(preds = model.reconstructions,
                          labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                          degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                          num_nodes = num_nodes,
                          pos_weight = pos_weight,
                          norm = norm,
                          clusters_distance = model.clusters)
    else:
        opt = OptimizerVAE(preds = model.reconstructions,
                           labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                           degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                           model = model,
                           num_nodes = num_nodes,
                           pos_weight = pos_weight,
                           norm = norm,
                           clusters_distance = model.clusters)
    if FLAGS.verbose:
        print("Done! \n")


    # Symmetrically normalized "message passing" matrices
    if FLAGS.verbose:
        print("Preprocessing on message passing matrices")
    adj_norm = preprocess_graph(adj + FLAGS.lamb*adj_louvain_init)
    adj_norm_layer2 = preprocess_graph(adj)
    if not FLAGS.fastgae:
        adj_label = sparse_to_tuple(adj + sp.eye(num_nodes))
    if FLAGS.verbose:
        print("Done! \n")


    # Degree matrices
    deg_matrix, deg_matrix_init = preprocess_degree(adj, FLAGS.simple)


    # Initialize TF session
    if FLAGS.verbose:
        print("Initializing TF session")
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    if FLAGS.verbose:
        print("Done! \n")


    # Model training
    if FLAGS.verbose:
        print("STEP 2/3 - MODEL TRAINING \n")
        print("Starting training")

    for iter in range(FLAGS.iterations):

        # Flag to compute running time for each iteration
        t = time.time()

        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_norm_layer2, adj_label, features, deg_matrix, placeholders)
        if FLAGS.fastgae:
            # Update sampled subgraph and sampled Louvain matrix
            feed_dict.update({placeholders['sampled_nodes']: sampled_nodes})
            # New node sampling
            sampled_nodes, adj_label, adj_label_sparse, = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)

        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.cost_adj, opt.cost_mod],
                        feed_dict = feed_dict)

        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display information on the iteration
            print("Iteration:", '%04d' % (iter + 1), "Loss:", "{:.5f}".format(avg_cost),
                  "Time:", "{:.5f}".format(time.time() - t))

    if FLAGS.verbose:
        print("Done! \n")

    # Compute embedding vectors, for evaluation
    if FLAGS.verbose:
        print("STEP 3/3 - MODEL EVALUATION \n")
        print("Computing the final embedding vectors, for evaluation")
    emb = sess.run(model.z_mean, feed_dict = feed_dict)
    if FLAGS.verbose:
        print("Done! \n")

    # Test model: link prediction (classification edges/non-edges)

    if FLAGS.verbose:
        print("Testing: link prediction")
    # Get ROC and AP scores
    roc_score, ap_score = link_prediction(test_edges, test_edges_false, emb)
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)
    if FLAGS.verbose:
        print("Done! \n")

EXPERIMENTS ON MODEL 1 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 88 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.71464 Time: 0.41946
Iteration: 0002 Loss: 1.70685 Time: 0.19829
Iteration: 0003 Loss: 1.68361 Time: 0.20042
Iteration: 0004 Loss: 1.63546 Time: 0.19514
Iteration: 0005 Loss: 1.62812 Time: 0.20058
Iteration: 0006 Loss: 1.61413 Time: 0.20000
Iteration: 0007 Loss: 1.58619 Time: 0.19592
Iteration: 0008 Loss: 1.56906 Time: 0.19473
Iteration: 0009 Loss: 1.56394 Time: 0.18913
Iteration: 0010 Loss: 1.47852 Time: 0.20143
Iteration: 0011 Lo

Iteration: 0176 Loss: 0.58976 Time: 0.18412
Iteration: 0177 Loss: 0.58526 Time: 0.18519
Iteration: 0178 Loss: 0.59000 Time: 0.18872
Iteration: 0179 Loss: 0.58635 Time: 0.18807
Iteration: 0180 Loss: 0.58751 Time: 0.18729
Iteration: 0181 Loss: 0.58561 Time: 0.19054
Iteration: 0182 Loss: 0.57999 Time: 0.18390
Iteration: 0183 Loss: 0.58093 Time: 0.19272
Iteration: 0184 Loss: 0.58419 Time: 0.18649
Iteration: 0185 Loss: 0.57750 Time: 0.19115
Iteration: 0186 Loss: 0.57773 Time: 0.18542
Iteration: 0187 Loss: 0.57751 Time: 0.17567
Iteration: 0188 Loss: 0.57732 Time: 0.18506
Iteration: 0189 Loss: 0.57610 Time: 0.19555
Iteration: 0190 Loss: 0.57421 Time: 0.18642
Iteration: 0191 Loss: 0.57170 Time: 0.18699
Iteration: 0192 Loss: 0.57248 Time: 0.18858
Iteration: 0193 Loss: 0.57034 Time: 0.19014
Iteration: 0194 Loss: 0.57016 Time: 0.18930
Iteration: 0195 Loss: 0.56836 Time: 0.18797
Iteration: 0196 Loss: 0.56671 Time: 0.19115
Iteration: 0197 Loss: 0.56802 Time: 0.18797
Iteration: 0198 Loss: 0.56146 Ti

Iteration: 0148 Loss: 0.62261 Time: 0.18994
Iteration: 0149 Loss: 0.62103 Time: 0.19054
Iteration: 0150 Loss: 0.61758 Time: 0.19136
Iteration: 0151 Loss: 0.61547 Time: 0.18745
Iteration: 0152 Loss: 0.61512 Time: 0.18649
Iteration: 0153 Loss: 0.61098 Time: 0.18964
Iteration: 0154 Loss: 0.61685 Time: 0.19153
Iteration: 0155 Loss: 0.61258 Time: 0.18690
Iteration: 0156 Loss: 0.61043 Time: 0.19133
Iteration: 0157 Loss: 0.61111 Time: 0.18897
Iteration: 0158 Loss: 0.60721 Time: 0.18089
Iteration: 0159 Loss: 0.60914 Time: 0.18307
Iteration: 0160 Loss: 0.60562 Time: 0.18577
Iteration: 0161 Loss: 0.61006 Time: 0.18800
Iteration: 0162 Loss: 0.60202 Time: 0.18516
Iteration: 0163 Loss: 0.60536 Time: 0.20192
Iteration: 0164 Loss: 0.60206 Time: 0.18902
Iteration: 0165 Loss: 0.60160 Time: 0.19748
Iteration: 0166 Loss: 0.59920 Time: 0.19650
Iteration: 0167 Loss: 0.59540 Time: 0.19873
Iteration: 0168 Loss: 0.59285 Time: 0.19207
Iteration: 0169 Loss: 0.59598 Time: 0.18235
Iteration: 0170 Loss: 0.59463 Ti

Iteration: 0119 Loss: 0.67292 Time: 0.18869
Iteration: 0120 Loss: 0.67525 Time: 0.18876
Iteration: 0121 Loss: 0.67766 Time: 0.17993
Iteration: 0122 Loss: 0.67090 Time: 0.18983
Iteration: 0123 Loss: 0.66999 Time: 0.18982
Iteration: 0124 Loss: 0.66428 Time: 0.18976
Iteration: 0125 Loss: 0.66902 Time: 0.18625
Iteration: 0126 Loss: 0.65871 Time: 0.18930
Iteration: 0127 Loss: 0.65961 Time: 0.18525
Iteration: 0128 Loss: 0.66095 Time: 0.18446
Iteration: 0129 Loss: 0.66139 Time: 0.19107
Iteration: 0130 Loss: 0.65441 Time: 0.18404
Iteration: 0131 Loss: 0.65000 Time: 0.19517
Iteration: 0132 Loss: 0.65709 Time: 0.18225
Iteration: 0133 Loss: 0.64624 Time: 0.19408
Iteration: 0134 Loss: 0.64395 Time: 0.19314
Iteration: 0135 Loss: 0.64578 Time: 0.18993
Iteration: 0136 Loss: 0.63977 Time: 0.19248
Iteration: 0137 Loss: 0.64278 Time: 0.19117
Iteration: 0138 Loss: 0.64398 Time: 0.18803
Iteration: 0139 Loss: 0.63727 Time: 0.18339
Iteration: 0140 Loss: 0.64150 Time: 0.17909
Iteration: 0141 Loss: 0.63485 Ti

Iteration: 0090 Loss: 0.77555 Time: 0.18633
Iteration: 0091 Loss: 0.76714 Time: 0.18321
Iteration: 0092 Loss: 0.76616 Time: 0.19119
Iteration: 0093 Loss: 0.75664 Time: 0.18824
Iteration: 0094 Loss: 0.74590 Time: 0.18829
Iteration: 0095 Loss: 0.75463 Time: 0.18635
Iteration: 0096 Loss: 0.74858 Time: 0.19096
Iteration: 0097 Loss: 0.74309 Time: 0.19220
Iteration: 0098 Loss: 0.74087 Time: 0.18888
Iteration: 0099 Loss: 0.73476 Time: 0.18343
Iteration: 0100 Loss: 0.72862 Time: 0.18075
Iteration: 0101 Loss: 0.72870 Time: 0.18703
Iteration: 0102 Loss: 0.72864 Time: 0.18953
Iteration: 0103 Loss: 0.71877 Time: 0.19036
Iteration: 0104 Loss: 0.71383 Time: 0.18782
Iteration: 0105 Loss: 0.71582 Time: 0.18899
Iteration: 0106 Loss: 0.71533 Time: 0.18714
Iteration: 0107 Loss: 0.71220 Time: 0.18618
Iteration: 0108 Loss: 0.70863 Time: 0.19077
Iteration: 0109 Loss: 0.69767 Time: 0.18371
Iteration: 0110 Loss: 0.70053 Time: 0.18676
Iteration: 0111 Loss: 0.69973 Time: 0.18615
Iteration: 0112 Loss: 0.69098 Ti

...

Iteration: 0005 Loss: 1.61220 Time: 0.19478
Iteration: 0006 Loss: 1.59296 Time: 0.18077
Iteration: 0007 Loss: 1.56966 Time: 0.18371
Iteration: 0008 Loss: 1.56354 Time: 0.18688
Iteration: 0009 Loss: 1.50766 Time: 0.18960
Iteration: 0010 Loss: 1.50727 Time: 0.18874
Iteration: 0011 Loss: 1.47381 Time: 0.19174
Iteration: 0012 Loss: 1.49963 Time: 0.18693
Iteration: 0013 Loss: 1.46042 Time: 0.19280
Iteration: 0014 Loss: 1.41855 Time: 0.19100
Iteration: 0015 Loss: 1.41456 Time: 0.19084
Iteration: 0016 Loss: 1.40125 Time: 0.18555
Iteration: 0017 Loss: 1.39765 Time: 0.18504
Iteration: 0018 Loss: 1.37547 Time: 0.18623
Iteration: 0019 Loss: 1.37574 Time: 0.18387
Iteration: 0020 Loss: 1.33655 Time: 0.19763
Iteration: 0021 Loss: 1.32086 Time: 0.18944
Iteration: 0022 Loss: 1.29362 Time: 0.18594
Iteration: 0023 Loss: 1.28779 Time: 0.19409
Iteration: 0024 Loss: 1.28090 Time: 0.18501
Iteration: 0025 Loss: 1.26687 Time: 0.19234
Iteration: 0026 Loss: 1.25140 Time: 0.19237
Iteration: 0027 Loss: 1.24521 Ti

Iteration: 0193 Loss: 0.57104 Time: 0.19075
Iteration: 0194 Loss: 0.57283 Time: 0.19004
Iteration: 0195 Loss: 0.57322 Time: 0.19044
Iteration: 0196 Loss: 0.57032 Time: 0.18993
Iteration: 0197 Loss: 0.56717 Time: 0.18713
Iteration: 0198 Loss: 0.56982 Time: 0.18971
Iteration: 0199 Loss: 0.56407 Time: 0.19072
Iteration: 0200 Loss: 0.56697 Time: 0.19437
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 

EXPERIMENTS ON MODEL 8 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 95 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TR

Iteration: 0164 Loss: 0.60350 Time: 0.18932
Iteration: 0165 Loss: 0.59925 Time: 0.18549
Iteration: 0166 Loss: 0.60028 Time: 0.18565
Iteration: 0167 Loss: 0.59855 Time: 0.19528
Iteration: 0168 Loss: 0.60165 Time: 0.18450
Iteration: 0169 Loss: 0.59809 Time: 0.19517
Iteration: 0170 Loss: 0.59526 Time: 0.19198
Iteration: 0171 Loss: 0.59001 Time: 0.18574
Iteration: 0172 Loss: 0.60045 Time: 0.18481
Iteration: 0173 Loss: 0.59566 Time: 0.18718
Iteration: 0174 Loss: 0.58854 Time: 0.18943
Iteration: 0175 Loss: 0.59092 Time: 0.19119
Iteration: 0176 Loss: 0.58973 Time: 0.18806
Iteration: 0177 Loss: 0.58885 Time: 0.18654
Iteration: 0178 Loss: 0.58367 Time: 0.18281
Iteration: 0179 Loss: 0.58823 Time: 0.18991
Iteration: 0180 Loss: 0.58318 Time: 0.18314
Iteration: 0181 Loss: 0.58229 Time: 0.19282
Iteration: 0182 Loss: 0.58297 Time: 0.18706
Iteration: 0183 Loss: 0.58507 Time: 0.19222
Iteration: 0184 Loss: 0.58034 Time: 0.18465
Iteration: 0185 Loss: 0.58226 Time: 0.18825
Iteration: 0186 Loss: 0.57946 Ti

Iteration: 0135 Loss: 0.64140 Time: 0.19374
Iteration: 0136 Loss: 0.64791 Time: 0.18784
Iteration: 0137 Loss: 0.63747 Time: 0.19159
Iteration: 0138 Loss: 0.63586 Time: 0.18835
Iteration: 0139 Loss: 0.63145 Time: 0.18738
Iteration: 0140 Loss: 0.63769 Time: 0.19275
Iteration: 0141 Loss: 0.63405 Time: 0.19099
Iteration: 0142 Loss: 0.63396 Time: 0.18476
Iteration: 0143 Loss: 0.63072 Time: 0.18785
Iteration: 0144 Loss: 0.62697 Time: 0.18657
Iteration: 0145 Loss: 0.62955 Time: 0.18767
Iteration: 0146 Loss: 0.62003 Time: 0.19131
Iteration: 0147 Loss: 0.62581 Time: 0.18315
Iteration: 0148 Loss: 0.62522 Time: 0.19491
Iteration: 0149 Loss: 0.62180 Time: 0.19660
Iteration: 0150 Loss: 0.62161 Time: 0.19212
Iteration: 0151 Loss: 0.61935 Time: 0.18340
Iteration: 0152 Loss: 0.62192 Time: 0.18863
Iteration: 0153 Loss: 0.61633 Time: 0.18771
Iteration: 0154 Loss: 0.61283 Time: 0.19291
Iteration: 0155 Loss: 0.61668 Time: 0.18366
Iteration: 0156 Loss: 0.61046 Time: 0.18299
Iteration: 0157 Loss: 0.60899 Ti

Iteration: 0107 Loss: 0.71392 Time: 0.19182
Iteration: 0108 Loss: 0.70788 Time: 0.19293
Iteration: 0109 Loss: 0.69806 Time: 0.19015
Iteration: 0110 Loss: 0.70667 Time: 0.19052
Iteration: 0111 Loss: 0.70393 Time: 0.18546
Iteration: 0112 Loss: 0.70105 Time: 0.19000
Iteration: 0113 Loss: 0.69146 Time: 0.18943
Iteration: 0114 Loss: 0.69331 Time: 0.18859
Iteration: 0115 Loss: 0.68935 Time: 0.18978
Iteration: 0116 Loss: 0.68479 Time: 0.19009
Iteration: 0117 Loss: 0.68080 Time: 0.18527
Iteration: 0118 Loss: 0.67792 Time: 0.18799
Iteration: 0119 Loss: 0.68095 Time: 0.19072
Iteration: 0120 Loss: 0.67843 Time: 0.19393
Iteration: 0121 Loss: 0.67618 Time: 0.18599
Iteration: 0122 Loss: 0.67728 Time: 0.18886
Iteration: 0123 Loss: 0.66405 Time: 0.18581
Iteration: 0124 Loss: 0.67188 Time: 0.18953
Iteration: 0125 Loss: 0.66269 Time: 0.18197
Iteration: 0126 Loss: 0.66607 Time: 0.18570
Iteration: 0127 Loss: 0.65915 Time: 0.20073
Iteration: 0128 Loss: 0.65529 Time: 0.18994
Iteration: 0129 Loss: 0.65532 Ti

In [5]:
# Report final results
print("FINAL RESULTS \n")

print('Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on', FLAGS.dataset)

print("All scores reported below are computed over the", FLAGS.nb_run, "run(s) \n")

print("Link prediction:\n")
print("Mean AUC score: ", np.mean(mean_roc))
print("Std of AUC scores: ", np.std(mean_roc), "\n")

print("Mean AP score: ", np.mean(mean_ap))
print("Std of AP scores: ", np.std(mean_ap), "\n \n")

FINAL RESULTS 

Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on wiki
All scores reported below are computed over the 10 run(s) 

Link prediction:

Mean AUC score:  0.9111491192088623
Std of AUC scores:  0.005777872555465969 

Mean AP score:  0.9344324138944022
Std of AP scores:  0.0036641922082108836 
 

